In [1]:
import ballchasing
import pandas as pd
import requests
import csv
import time
import os

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm

from bs4 import BeautifulSoup
pd.set_option('display.max_colwidth', 500)

#set up the api
api = ballchasing.Api(os.getenv('API_KEY'))

#selenium function to pull values from player RL Tracker pages
def pull_player_stat(url, selectors, by="xpath", headless=False):
    # Setup Chrome
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless=new")  # Updated headless argument for newer Chrome versions
    #chrome_options.add_argument("--disable-gpu")  # Recommended for headless mode
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    
    # Load the webpage
    driver.get(url)
    
    try:
        time.sleep(3)
        # Wait for the element with your XPath
        wait = WebDriverWait(driver, 7)  # Wait up to 20 seconds

        extracted_values = {}
        
        for key, selector in selectors.items():
            try:
                element = wait.until(EC.presence_of_element_located((By.XPATH, selector)))

                # Get the text content of the element
                value = element.get_attribute('textContent')  # Extract textContent
                value = value.replace(',', '')  # Remove commas if needed
                value = int(value)  # Convert to integer (if required)
                extracted_values[key] = value #store in the dict

            except Exception as e:
                extracted_values[key] = None  # Append None if there's an issue with this specific selector

        return extracted_values
    
    except Exception as e:
        print("Error:", e)
        driver.save_screenshot('error_screenshot.png')  # Save a screenshot for debugging
    
    finally:
        driver.quit()

In [2]:
#setting xpaths for various stats
player_wins_path = '/html/body/div/div/div[2]/div[3]/div/main/div[3]/div[2]/div[1]/aside/div[3]/div/div[1]/div/div[2]/span[2]/span'
player_mvps_path = '/html/body/div/div/div[2]/div[3]/div/main/div[3]/div[2]/div[1]/aside/div[3]/div/div[7]/div/div[2]/span[2]/span'
player_peak_mmr_path = '/html/body/div/div/div[2]/div[3]/div/main/div[3]/div[2]/div[1]/div/div/div[1]/div[2]/table/tbody/tr[3]/td[3]/div/div[2]/div[1]/div'


#joining them into a list of selectors
selectors = {'player_wins' : player_wins_path,
             'player_mvps': player_mvps_path,
            'player_peak_mmr': player_peak_mmr_path}

In [3]:
#bring in full list of replay IDs
with open('replay_ids', 'r') as file:
    csv_reader = csv.reader(file)
    replay_list = list(csv_reader)[0]

replay_stats =[]

In [4]:
#bring in full list of replay IDs
#prev_pulls = pd.read_csv('replay_data_1.csv')
#finished = list(prev_pulls['replay_id'])
#print(len(finished))

In [5]:
print(len(replay_list))

1386


In [6]:
#replay_list = [id for id in replay_list if id not in finished]
#print(len(replay_list))

In [7]:
for i in tqdm(replay_list):
    
    # Get a specific replay
    try:
        replay = api.get_replay(i)
    except Exception as e:
        print(f"Error accessing replay {i}: {str(e)}")
        print(f"API key used: {api.api_key}")
        continue
    
    players = []
    colors = ['blue','orange']
    player_num = [0,1]
    
    for x in colors:
        for y in player_num:
            player_name = replay[x]['players'][y]['name']
            if player_name == 'thecomebackkid13':
                player_name = 'MiniSempervirens'
            player_platform = replay[x]['players'][y]['id']['platform']
            if player_platform == 'steam':
                tracker_url = 'https://rocketleague.tracker.network/rocket-league/profile/' + player_platform + '/' + replay[x]['players'][y]['id']['id']
            elif player_platform == 'xbox':
                tracker_url = 'https://rocketleague.tracker.network/rocket-league/profile/xbl/' + player_name
            elif player_platform == 'ps4':
                tracker_url = 'https://rocketleague.tracker.network/rocket-league/profile/psn/' + player_name
            elif player_platform == 'ps5':
                tracker_url = 'https://rocketleague.tracker.network/rocket-league/profile/psn/' + player_name
            else:
                tracker_url = 'https://rocketleague.tracker.network/rocket-league/profile/' + player_platform + '/' + player_name
        
            #values = pull_player_stat(tracker_url,selectors)
            
            players.append({'team' : x,
                            'player': player_name,
                            'platform': player_platform,
                            'url': tracker_url})
                            #'player_wins': values['player_wins'],
                            #'player_mvps': values['player_mvps'],
                            #'player_peak_mmr': values['player_peak_mmr']})
    
    player_df = pd.DataFrame(players)
    
    my_team_ind = player_df[player_df['player']=='MiniSempervirens']['team'].reset_index(drop=True)
    
    #contstruct a df of just my team
    my_team = player_df[player_df['team'] == my_team_ind[0]]
    
    #contstruct a df of other team
    opp_team = player_df[player_df['team'] != my_team_ind[0]].reset_index(drop=True)
    
    #creating a df containing just my teammate
    tm_df = my_team[(my_team['player']!='MiniSempervirens') & (my_team['player']!='thecomebackkid13')].reset_index(drop=True)

    tm8_name = tm_df['player'][0]
    opp_name_1 = opp_team['player'][0]
    opp_name_2 = opp_team['player'][1]   
    opp_url_1 = opp_team['url'][0]
    opp_url_2 = opp_team['url'][1]
    tm8_url = tm_df['url'][0]
    #creating teammate metrics
    #tm_wins = tm_df['player_wins'].min()
    #tm_mvps = tm_df['player_mvps'].min()
    #tm_peak_mmr = tm_df['player_peak_mmr'].min()
    
    #min and diff stats for each category
    #opp_min_wins = opp_team['player_wins'].min()
    #opp_diff_wins = opp_team['player_wins'].max() - opp_team['player_wins'].min()
    
    #opp_min_mvps = opp_team['player_mvps'].min()
    #opp_diff_mvps = opp_team['player_mvps'].max() - opp_team['player_mvps'].min()
    
    #opp_min_peak_mmr = opp_team['player_peak_mmr'].min()
    #opp_diff_peak_mmr = opp_team['player_peak_mmr'].max() - opp_team['player_peak_mmr'].min()
    
    #save my team color
    tm_color = my_team_ind[0]
    #log id
    replay_id = replay['id']
    #determine win or loss
    game_win = replay['title'][-3:]=='win'
    #is overtime
    is_ot = replay['overtime']
    
    #pull out my game data
    my_data = [player for player in replay[tm_color]['players'] if (player['name'] == 'MiniSempervirens') | (player['name'] == 'thecomebackkid13')]
    
    #general game stats
    game_time = my_data[0]['end_time']
    car_id = my_data[0]['car_id']
    steer_sens = my_data[0]['steering_sensitivity']
    
    #boost stats
    boost_per_min = my_data[0]['stats']['boost']['bpm']
    boost_avg_amt = my_data[0]['stats']['boost']['avg_amount']
    big_stoln_per_min = my_data[0]['stats']['boost']['count_stolen_big']/(game_time/60)
    big_clctd_per_min = my_data[0]['stats']['boost']['count_collected_big']/(game_time/60)
    sml_clctd_per_min = my_data[0]['stats']['boost']['count_collected_small']/(game_time/60)
    pct_zero_boost = my_data[0]['stats']['boost']['percent_zero_boost']
    pct_full_boost = my_data[0]['stats']['boost']['percent_full_boost']
    pct_0_25_boost = my_data[0]['stats']['boost']['percent_boost_0_25']
    pct_25_50_boost = my_data[0]['stats']['boost']['percent_boost_25_50']
    pct_75_100_boost = my_data[0]['stats']['boost']['percent_boost_75_100']
    
    #movement stats
    avg_speed = my_data[0]['stats']['movement']['avg_speed']
    pct_supersonic = my_data[0]['stats']['movement']['percent_supersonic_speed']
    pct_slow = my_data[0]['stats']['movement']['percent_slow_speed']
    avg_powerslide_duration = my_data[0]['stats']['movement']['avg_powerslide_duration']
    powerslide_per_min = my_data[0]['stats']['movement']['count_powerslide']/(game_time/60)
    pct_ground = my_data[0]['stats']['movement']['percent_ground']
    pct_high_air = my_data[0]['stats']['movement']['percent_high_air']
    
    #positioning stats
    avg_dist_to_ball = my_data[0]['stats']['positioning']['avg_distance_to_ball']
    avg_dist_to_mates = my_data[0]['stats']['positioning']['avg_distance_to_mates']
    pct_def_third = my_data[0]['stats']['positioning']['percent_defensive_third']
    pct_off_third = my_data[0]['stats']['positioning']['percent_offensive_third']
    pct_behind_ball = my_data[0]['stats']['positioning']['percent_behind_ball']
    pct_most_back = my_data[0]['stats']['positioning']['percent_most_back']
    percent_closest_to_ball = my_data[0]['stats']['positioning']['percent_closest_to_ball']
    
    #demo stats
    demos_given_per_min = my_data[0]['stats']['demo']['inflicted']/(game_time/60)
    demos_taken_per_min = my_data[0]['stats']['demo']['taken']/(game_time/60)

    row = {
        'replay_id': replay_id,
        'game_win': game_win,
        'is_ot': is_ot,
        'tm_color': tm_color,
        'game_time': game_time,
        'car_id': car_id,
        'steer_sens': steer_sens,
        'tm8_name': tm8_name,
        'opp_name_1': opp_name_1,
        'opp_name_2': opp_name_2,
        'opp_url_1': opp_url_1,
        'opp_url_2': opp_url_2,
        'tm8_url': tm8_url,
        #'tm8_wins': tm_wins,
        #'tm8_mvps': tm_mvps,
        #'tm8_peak_mmr': tm_peak_mmr,
        #'opp_min_wins': opp_min_wins,
        #'opp_diff_wins': opp_diff_wins,
        #'opp_min_mvps': opp_min_mvps,
        #'opp_diff_mvps': opp_diff_mvps,
        #'opp_min_peak_mmr': opp_min_peak_mmr,
        #'opp_diff_peak_mmr': opp_diff_peak_mmr,
        'boost_per_min': boost_per_min,
        'boost_avg_amt': boost_avg_amt,
        'big_stoln_per_min': big_stoln_per_min,
        'big_clctd_per_min': big_clctd_per_min,
        'sml_clctd_per_min': sml_clctd_per_min,
        'pct_zero_boost': pct_zero_boost,
        'pct_full_boost': pct_full_boost,
        'pct_0_25_boost': pct_0_25_boost,
        'pct_25_50_boost': pct_25_50_boost,
        'pct_75_100_boost': pct_75_100_boost,
        'avg_speed': avg_speed,
        'pct_supersonic': pct_supersonic,
        'pct_slow': pct_slow,
        'avg_powerslide_duration': avg_powerslide_duration,
        'powerslide_per_min': powerslide_per_min,
        'pct_ground': pct_ground,
        'pct_high_air': pct_high_air,
        'avg_dist_to_ball': avg_dist_to_ball,
        'avg_dist_to_mates': avg_dist_to_mates,
        'pct_def_third': pct_def_third,
        'pct_off_third': pct_off_third,
        'pct_behind_ball': pct_behind_ball,
        'pct_most_back': pct_most_back,
        'percent_closest_to_ball': percent_closest_to_ball,
        'demos_given_per_min': demos_given_per_min,
        'demos_taken_per_min': demos_taken_per_min
    }
    replay_stats.append(row)

df = pd.DataFrame(replay_stats)
df

100%|██████████████████████████████████████████████████████████████████████████████| 1386/1386 [47:28<00:00,  2.06s/it]


,replay_id,game_win,is_ot,tm_color,game_time,car_id,steer_sens,tm8_name,opp_name_1,opp_name_2,...,pct_high_air,avg_dist_to_ball,avg_dist_to_mates,pct_def_third,pct_off_third,pct_behind_ball,pct_most_back,percent_closest_to_ball,demos_given_per_min,demos_taken_per_min
0,27f4cd7a-0e19-455e-bb90-a7863b0622ef,False,True,blue,388.96512,23,1.55,Stop Cryinggggg,KandyKaner20088,c0mplexMX,...,5.289062,2392,3386,54.678505,19.596863,68.236980,53.518578,49.462894,0.462766,0.154255
1,a360b058-2d64-45bc-a7a7-aff8f46a3bfb,False,False,blue,393.70996,23,1.55,DR34MS.,Green._n,miaduh,...,4.370624,2676,3332,55.423190,19.012346,71.648210,54.793400,49.584072,0.152396,0.000000
2,7339c913-0322-4d0a-9fae-8d417cd46713,False,True,orange,441.89780,23,1.55,Pyro,Grn.Day,*******,...,2.391483,2398,3527,51.660854,18.758408,66.935684,43.247868,51.374664,0.543112,0.135778
3,f109c1fc-31fd-4993-a4f7-19f467a1bc88,False,False,orange,43.83187,23,1.55,Envixity480,sponge,BestV7,...,0.000000,4099,3300,48.789950,28.378994,77.414940,54.454136,17.710812,1.368867,0.000000
4,13285fae-abed-4de2-812d-51b9c8e12875,False,True,orange,545.52940,23,1.55,Elixric,SAVAGE_1RL,Ezzcrankz,...,2.485631,2424,3382,41.891840,24.445772,68.088600,51.050190,52.187588,0.549925,0.329955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381,e1f46624-7ef2-4441-9fe5-aeb09c403046,False,False,orange,344.15076,4284,1.27,i can fix that,SixersJake1021,MushySquffins,...,0.214491,2693,3183,50.528885,16.430628,79.582184,59.049583,46.739777,0.174342,0.174342
1382,233608bc-92ed-46f3-9339-a9e1a5148d80,False,False,blue,232.33160,4284,1.27,i can fix that,xGEN Peck x8,OBD Mexxboy,...,0.120721,3016,3594,34.319220,26.976805,70.154790,40.252530,41.730100,0.258252,0.000000
1383,6df07200-6f5a-4e2d-b245-bbf7dc810da4,False,False,orange,171.41968,4284,1.27,i can fix that,OJ DA JUICE,Talion,...,1.590457,2844,3851,38.515965,30.540287,83.996025,50.003136,47.179870,0.350018,0.000000
1384,e82cc842-c583-4447-9e57-89c91028b80d,False,False,orange,344.67734,4284,1.00,OlyEnder,Dionisio,lil' chillin,...,0.369165,2686,3242,50.808094,22.306843,67.246090,52.887093,48.821186,0.174076,0.000000


In [8]:
df.to_csv('replay_data_2.csv',index=False)

In [9]:
df = pd.DataFrame(replay_stats)
df

,replay_id,game_win,is_ot,tm_color,game_time,car_id,steer_sens,tm8_name,opp_name_1,opp_name_2,...,pct_high_air,avg_dist_to_ball,avg_dist_to_mates,pct_def_third,pct_off_third,pct_behind_ball,pct_most_back,percent_closest_to_ball,demos_given_per_min,demos_taken_per_min
0,27f4cd7a-0e19-455e-bb90-a7863b0622ef,False,True,blue,388.96512,23,1.55,Stop Cryinggggg,KandyKaner20088,c0mplexMX,...,5.289062,2392,3386,54.678505,19.596863,68.236980,53.518578,49.462894,0.462766,0.154255
1,a360b058-2d64-45bc-a7a7-aff8f46a3bfb,False,False,blue,393.70996,23,1.55,DR34MS.,Green._n,miaduh,...,4.370624,2676,3332,55.423190,19.012346,71.648210,54.793400,49.584072,0.152396,0.000000
2,7339c913-0322-4d0a-9fae-8d417cd46713,False,True,orange,441.89780,23,1.55,Pyro,Grn.Day,*******,...,2.391483,2398,3527,51.660854,18.758408,66.935684,43.247868,51.374664,0.543112,0.135778
3,f109c1fc-31fd-4993-a4f7-19f467a1bc88,False,False,orange,43.83187,23,1.55,Envixity480,sponge,BestV7,...,0.000000,4099,3300,48.789950,28.378994,77.414940,54.454136,17.710812,1.368867,0.000000
4,13285fae-abed-4de2-812d-51b9c8e12875,False,True,orange,545.52940,23,1.55,Elixric,SAVAGE_1RL,Ezzcrankz,...,2.485631,2424,3382,41.891840,24.445772,68.088600,51.050190,52.187588,0.549925,0.329955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381,e1f46624-7ef2-4441-9fe5-aeb09c403046,False,False,orange,344.15076,4284,1.27,i can fix that,SixersJake1021,MushySquffins,...,0.214491,2693,3183,50.528885,16.430628,79.582184,59.049583,46.739777,0.174342,0.174342
1382,233608bc-92ed-46f3-9339-a9e1a5148d80,False,False,blue,232.33160,4284,1.27,i can fix that,xGEN Peck x8,OBD Mexxboy,...,0.120721,3016,3594,34.319220,26.976805,70.154790,40.252530,41.730100,0.258252,0.000000
1383,6df07200-6f5a-4e2d-b245-bbf7dc810da4,False,False,orange,171.41968,4284,1.27,i can fix that,OJ DA JUICE,Talion,...,1.590457,2844,3851,38.515965,30.540287,83.996025,50.003136,47.179870,0.350018,0.000000
1384,e82cc842-c583-4447-9e57-89c91028b80d,False,False,orange,344.67734,4284,1.00,OlyEnder,Dionisio,lil' chillin,...,0.369165,2686,3242,50.808094,22.306843,67.246090,52.887093,48.821186,0.174076,0.000000


In [10]:
tm_df

,team,player,platform,url
0,blue,DaddyBrickly,xbox,https://rocketleague.tracker.network/rocket-league/profile/xbl/DaddyBrickly


In [11]:
opp_team

,team,player,platform,url
0,orange,ROKT-LEEG,epic,https://rocketleague.tracker.network/rocket-league/profile/epic/ROKT-LEEG
1,orange,MiniMan7x,epic,https://rocketleague.tracker.network/rocket-league/profile/epic/MiniMan7x
